# Price Prediction of used cars - Pakwheels

-----------


## Import Libraries

In [408]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')



## Load Dataset

In [409]:
df = pd.read_csv('D:/DS Bootcamp/Machine Learning/Used cars Price prediction/Used-Cars-Price-Prediction-Pakwheels/Pakwheels_used_cars.csv')
df.head()

,title,location,model_year,fuel_type,transmission,km_driven,registered_in,color,assembly,engine_capacity,body_type,last_updated,price,url
0,Suzuki Mehran 2003,Multan Punjab,2003.0,Petrol,Manual,"170,000 km",Lahore,White,Local,800 cc,NaN,"May 07, 2025",PKR 5.8 Lakh,https://www.pakwheels.com/used-cars/suzuki-meh...
1,Mitsubishi Lancer GLX Automatic 1.6 2006,Lahore Punjab,2006.0,Petrol,Automatic,"119,080 km",Islamabad,Labrador Black Pearl,Imported,1600 cc,Sedan,"Apr 29, 2025",PKR 25 Lakh,https://www.pakwheels.com/used-cars/mitsubishi...
2,Suzuki Wagon R Stingray X 2013,"Abbot Road, Lahore Punjab",2013.0,Petrol,Automatic,"87,000 km",Punjab,Black,Imported,660 cc,Hatchback,"Apr 29, 2025",PKR 26.5 Lakh,https://www.pakwheels.com/used-cars/suzuki-wag...
3,Toyota Land Cruiser ZX 2019,Lahore Punjab,2019.0,Petrol,Automatic,"30,000 km",Un-Registered,White Pearl Crystal Shine,Imported,4600 cc,SUV,"Apr 28, 2025",PKR 4.63 crore,https://www.pakwheels.com/used-cars/toyota-lan...
4,Suzuki Alto VXL AGS 2021,"Rehmania Town, Faisalabad Punjab",2021.0,Petrol,Automatic,"15,800 km",Punjab,Silver,Local,660 cc,Hatchback,"Apr 30, 2025",PKR 27.7 Lakh,https://www.pakwheels.com/used-cars/suzuki-alt...


------

## Data Preprocessing

### Handle Data inconsisties

In [410]:
pd.set_option('display.max_rows', False)

From title column extract the brand name and store it to a new column called brand

In [411]:
df.insert(0, 'brand', df['title'].str.split().str[0])

From title remove the model yr bcz we already have year of model in dataset


In [412]:
df['title'] = df['title'].str.replace(r'\b\d{4}$', '', regex=True).str.strip()

Rename the title to model_name

In [413]:
df.rename(columns={'title':'model_name'},inplace=True)

Extract from location all the provinces and stored in new column province

In [414]:
df.insert(2, 'province', df['location'].str.split().str[-1])

Correct the dtype of model_year into int.

In [415]:
df['model_year'] = pd.to_numeric(df['model_year'], errors='coerce')

Replace comma and km from the km_driven column

In [416]:
df['km_driven'] = df['km_driven'].replace({',': '', ' km': ''}, regex=True).str.strip()

Correct the dtype of km_driven into int

In [417]:
df['km_driven'] = pd.to_numeric(df['km_driven'], errors='coerce')

Add a new column of engine size in numeric

In [418]:
df.insert(12, 'engine_power', df['engine_capacity'].str.split().str[0].astype('float'))

Remove those engine in cc less than 600

In [419]:
df['unit'] = df['engine_capacity'].str.extract(r'([a-zA-Z]+)')

df = df[~((df['unit'].str.lower() == 'cc') & (df['engine_power'] < 600))]
df.reset_index(drop=True, inplace=True)

Remove the unit column

In [420]:
df.drop(columns=['unit'], inplace=True)

We observed that some cars have Km_driven listed as 1 km, which is highly unlikely given that the car models range from 1990 to 2022. It's not realistic for 102 cars to have been driven only 1 km, so we consider this a data entry error. Since we don't have accurate information about their actual mileage, we decided to remove these rows from the dataset.

In [421]:
df = df[df['km_driven'] > 1]

Correct the last_updated column type into date format

In [422]:
df['last_updated'] = pd.to_datetime(df['last_updated'], format='%b %d, %Y', errors='coerce')

We have 180 rows where the price is listed as "on call for price." Since price is our target column, including any incorrect or missing information could negatively impact the performance of our model. Therefore, we have decided to remove these rows to ensure the quality and accuracy of the data.

In [423]:
df = df[df['price'] != 'Call for price']

Remove PKR from the price column

In [424]:
df['price'] = df['price'].str.replace('PKR', '', regex=True).str.strip()

Calculate the lacs and crores values and convert into float

In [425]:
def convert_price(value):
    if isinstance(value, (float, int)):
        return value  

    value = value.strip().lower()
    if 'crore' in value:
        num = value.replace('crore', '').strip()  
        try:
            return float(num) * 10_000_000
        except ValueError:
            return None
    elif 'lakh' in value:
        num = value.replace('lakh', '').strip() 
        try:
            return float(num) * 100_000  
        except ValueError:
            return None  
    try:
        return float(value)
    except ValueError:
        return None 

df['price'] = df['price'].apply(convert_price)

Drop the Url column bcz it didn't help in prediction

In [426]:
df.drop(columns=['url'], inplace=True)

We have too many unique colors, which could negatively affect our model. Therefore, I've decided to convert the colors into base colors for better generalization and model performance.

In [427]:
df['color'].nunique()

430

In [429]:
base_colors = ['white', 'silver', 'black', 'grey', 'blue', 'green', 'red', 'orange', 'pink', 'purple', 'indigo', 'brown', 'turquoise', 'titanium', 'metallic', 'maroon'
               'gold', 'bronze', 'beige', 'navy', 'burgundy','magneta', 'unlisted']

def map_to_base_color(color):
    color = color.lower()
    for base_color in base_colors:
        if base_color in color:
            return base_color
    return 'others' 

df['color'] = df['color'].apply(map_to_base_color)

from 430 to 22 color that reduces the model complexity

In [430]:
df['color'].nunique()

22

In [431]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 9698 entries, 0 to 9994
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   brand            9698 non-null   object        
 1   model_name       9698 non-null   object        
 2   province         9698 non-null   object        
 3   location         9698 non-null   object        
 4   model_year       8731 non-null   float64       
 5   fuel_type        9698 non-null   object        
 6   transmission     9698 non-null   object        
 7   km_driven        9698 non-null   float64       
 8   registered_in    9698 non-null   object        
 9   color            9698 non-null   object        
 10  assembly         9698 non-null   object        
 11  engine_capacity  9698 non-null   object        
 12  engine_power     9698 non-null   float64       
 13  body_type        8685 non-null   object        
 14  last_updated     9698 non-null   datetime64[n

,model_year,km_driven,engine_power,last_updated,price
count,8731.000000,9698.000000,9698.000000,9698,9.698000e+03
mean,2013.406368,94796.237781,1351.500555,2025-04-30 13:21:48.888430592,4.591775e+06
min,1990.000000,2.000000,4.000000,2025-03-25 00:00:00,1.100000e+05
25%,2008.000000,40000.000000,800.000000,2025-04-27 00:00:00,1.686250e+06
50%,2015.000000,81000.000000,1300.000000,2025-05-01 00:00:00,2.950000e+06
75%,2020.000000,125000.000000,1600.000000,2025-05-05 00:00:00,4.850000e+06
max,2022.000000,1000000.000000,11136.000000,2025-05-08 00:00:00,1.775000e+08
std,7.290992,88690.723116,725.475523,NaN,7.207433e+06
